# Object Tracking

In [8]:
import cv2 
import numpy as np

In [10]:
# set some parameter values using Shi-Tomasi Corner Detection
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize =7)


In [11]:
# smaller window size are more prone to noise, larger windown
# size may not be sensetive to smaller motions of the points, points
# may be moving but it feels the frame is not moving
# maxLevel creates image pyramid with different resolution
# next we use pip operator providing two criteria to perform Lucas-Kanade optical Flow
# provide max iterations and provide epsilon which is 0.03. More iteration
# means more exhaustive serach of points, smaller eps means gonna finish earlier
# esp exchange speed of your tracking vs accuracy of your tracking
lk_params = dict(winSize = (200, 200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [14]:
import cv2
import numpy as np

# Initialize the video capture
cap = cv2.VideoCapture(0)

# Read the very first frame
ret, prev_frame = cap.read()

# Convert the first frame to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Points to track using good feature tracking
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Create an empty mask
mask = np.zeros_like(prev_frame)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

while True:
    ret, frame = cap.read()

    # Convert the current frame to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)

    # Filter out good points
    good_new = nextPts[status == 1]
    good_prev = prevPts[status == 1]

    # Draw lines and circles
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()

        mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_prev), int(y_prev)), (0, 255, 0), 3)
        frame = cv2.circle(frame, (int(x_new), int(y_new)), 8, (0, 0, 255), -1)

    # Combine frame and mask
    img = cv2.add(frame, mask)

    # Display the result
    cv2.imshow('Tracking', img)

    # Exit on 'Esc' key press
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break

    # Update previous frame and points
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)

# Release video capture and destroy windows
cv2.destroyAllWindows()
cap.release()

# Optical Flow with OpenCV Part2 Dense Optical Flow

In [17]:
capture = cv2.VideoCapture(0)

ret, frame1 = capture.read()

prevImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)

hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = capture.read()

    nextImg = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # assigning value for window size, frame, and iteration etc
    flow =cv2.calcOpticalFlowFarneback(prevImg, nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv2.cartToPolar(flow[:, :, 0], flow[:,:,1], angleInDegrees=True)


    hsv_mask[:,:,0] = ang/2

    hsv_mask[:,:, 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

    bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
    cv2.imshow('frame', bgr)


    k = cv2.waitKey(10) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()
capture.release()

# MeanShift and CamShift Tracking

In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture(0)

ret, frame = cap.read()

# objection first place
face_cascade = cv2.CascadeClassifier("filepath")
face_rects = face_cascade.detectMultiScale(frame)

(face_x, face_y, w, h) = tuple(face_rects[0])

track_window = (face_x, face_y, x, y)

# drawing region of interest
roi = frame[face_y:face_y+h, face_x, face_x+w]

#hsv color maping
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# find histogram to back project target on each frame to calculate mean shift
roi_hist = cv2.calcHist([hsv_roi], [0],  None, [180], [0, 180])

# Normalize histograme value
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# setting up termination criteria
term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()

    if ret == True:

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # calculate back projection based on roi_hist we created earlier
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        ret, track_window = cv2.meanSHift(dst, track_window, term_criteria)


        # draw new window based on updated track_windown on new image
        x, y, w, h = track_window
        img2 = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255))

        cv2.imshow('img', img2)

        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
    else:
        break


cv2.destroyAllWindows()
cap.release()


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'


In [ ]:
cap = cv2.VideoCapture(0)

ret, frame = cap.read()

# objection first place
face_cascade = cv2.CascadeClassifier("filepath")
face_rects = face_cascade.detectMultiScale(frame)

(face_x, face_y, w, h) = tuple(face_rects[0])

track_window = (face_x, face_y, x, y)

# drawing region of interest
roi = frame[face_y:face_y+h, face_x, face_x+w]

#hsv color maping
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# find histogram to back project target on each frame to calculate mean shift
roi_hist = cv2.calcHist([hsv_roi], [0],  None, [180], [0, 180])

# Normalize histograme value
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# setting up termination criteria
term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()

    if ret == True:

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # calculate back projection based on roi_hist we created earlier
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        ret, track_window = cv2.CamShift(dst, track_window, term_criteria)

        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame, [pts], True, (0, 0, 255), 5)

    

        # draw new window based on updated track_windown on new image
        x, y, w, h = track_window
        img2 = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255))

        cv2.imshow('img', img2)

        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
    else:
        break


cv2.destroyAllWindows()
cap.release()

# Tracking APIs with OpenCV

In [5]:
import cv2

def ask_for_tracker():
    print('Welcome! What tracker API would you like to use?')
    print('Enter 0 for BOOSTING: ')
    print('Enter 1 for MIL: ')
    print('Enter 2 for KCF: ')
    print('Enter 3 for TLD: ')
    print('Enter 4 for MEDIANFLOW: ')

    choice = input('Please select your tracker: ')

    if choice == '0':
        tracker = cv2.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.TrackerTLD_create()
    if choice == '4':
        tracker = cv2.TrackerMedianFlow_create()

    return tracker

tracker = ask_for_tracker()


Welcome! What tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


UnboundLocalError: cannot access local variable 'tracker' where it is not associated with a value

In [6]:
import cv2

def ask_for_tracker():
    print('Welcome! What tracker API would you like to use?')
    print('Enter 0 for BOOSTING: ')
    print('Enter 1 for MIL: ')
    print('Enter 2 for KCF: ')
    print('Enter 3 for TLD: ')
    print('Enter 4 for MEDIANFLOW: ')

    while True:
        choice = input('Please select your tracker: ')
        if choice in ['0', '1', '2', '3', '4']:
            break
        else:
            print('Invalid input. Please enter a number between 0 and 4.')

    if choice == '0':
        tracker = cv2.TrackerBoosting_create()
    elif choice == '1':
        tracker = cv2.TrackerMIL_create()
    elif choice == '2':
        tracker = cv2.TrackerKCF_create()
    elif choice == '3':
        tracker = cv2.TrackerTLD_create()
    elif choice == '4':
        tracker = cv2.TrackerMedianFlow_create()

    return tracker

tracker = ask_for_tracker()


Welcome! What tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 
Invalid input. Please enter a number between 0 and 4.
Invalid input. Please enter a number between 0 and 4.
Invalid input. Please enter a number between 0 and 4.
Invalid input. Please enter a number between 0 and 4.
Invalid input. Please enter a number between 0 and 4.


In [1]:
import cv2

def ask_for_tracker():
    tracker = None  # Initialize tracker with a default value
    print('Welcome! What tracker API would you like to use?')
    print('Enter 0 for BOOSTING: ')
    print('Enter 1 for MIL: ')
    print('Enter 2 for KCF: ')
    print('Enter 3 for TLD: ')
    print('Enter 4 for MEDIANFLOW: ')

    while True:
        choice = input('Please select your tracker: ')
        if choice in ['0', '1', '2', '3', '4']:
            break
        else:
            print('Invalid input. Please enter a number between 0 and 4.')

    if choice == '0':
        tracker = cv2.TrackerBoosting_create()
    elif choice == '1':
        tracker = cv2.TrackerMIL_create()
    elif choice == '2':
        tracker = cv2.TrackerKCF_create()
    elif choice == '3':
        tracker = cv2.TrackerTLD_create()
    elif choice == '4':
        tracker = cv2.TrackerMedianFlow_create()

    return tracker

tracker = ask_for_tracker()


Welcome! What tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


In [2]:
tracker

< cv2.TrackerMIL 00000181891BA030>

In [5]:
str(tracker).split()[0][1:] # remove less than sign

''

In [6]:
tracker_name = str(tracker).split()[0][1:] # remove less than sign

In [8]:
tracker = ask_for_tracker()

str(tracker).split()[0][1:] # remove less than sign

# read video
cap = cv2.VideoCapture(0)

# read first frame
ret, frame =cap.read()
# sepcial function allows to draw on the first frame the desigr RoI
roi = cv2.selectROI(frame, False)

#initilizing tracker with first frame and draw bounding box
ret = tracker.init(frame, roi)

while True:
    # keep reading new frames
    ret, frame = cap.read()

    # roi variable is a tuple of 4 flats, convet to int
    (x, y, w, h) = tuple(map(int, roi))

    # draw rectanlge as tracker moves
    if success:
        # tracking success
        p1 = (x, y)
        p2 = (x+w, y+h)

        cv2.rectangle(frame, p1, p2, (0, 255, 0), 3)

    else:
        # tracker is failure
        cv2.putText(frame,"Failure", (100, 200), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1)

    cv2.putText(frame, tracker_name, (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255,0), 3)

    cv2.imshow(tracker_name, frame)


    k = cv2.waitKey(2) ^ 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()





Welcome! What tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\video\src\tracking\detail\tracking_feature.cpp:128: error: (-215:Assertion failed) !patchSize.empty() in function 'cv::detail::tracking::feature::CvHaarEvaluator::FeatureHaar::generateRandomFeature'


: 